# Identication of putative contaminant taxa

In [1]:
# tempdir <- function() {return("~/.Rtmp")}

In [1]:
setwd("/mnt/c/Users/Cedric/Desktop/git_repos/blood_microbiome")
require(tidyverse)
require(ggplot2)
require(data.table)
require(ANCOMBC)
require(phyloseq)

Loading required package: tidyverse

Warning message in system("timedatectl", intern = TRUE):
“running command 'timedatectl' had status 1”
── Attaching packages ─────────────────────────────────────────────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.3     ✔ dplyr   1.0.7
✔ tidyr   1.1.3     ✔ stringr 1.4.0
✔ readr   2.0.1     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Loading required package: data.table


Attaching package: ‘data.table’


The following objects are masked from ‘package:dplyr’:

    between, first, last


The following object is masked from ‘package:purrr’:

    transpose


Loading required package: ANCOMBC

Loading required package: phyloseq



### Create Phyloseq object

#### Load abundance matrix

In [3]:
load_data <- function(file_path) {
    df <- fread(file_path) %>%
        separate(sample, into = c(NA, "npm_research_id"), sep = "\\.")
    return(df)
}

df_to_phy <- function(df) {
    df <- as.matrix(df, rownames=1)
    df <- otu_table(df, taxa_are_rows = F)
    return(df)
}

df <- load_data(str_glue("data/temp_files_9999/07_abundance_matrix/abundance_matrix.subset_9999.S.tsv"))

#### Load metadata

In [4]:
load_metadata <- function(file_path, df) {
    meta <- fread(file_path, na.strings=c("", NA))
    meta <- meta %>% 
        filter(npm_research_id %in% df$npm_research_id) %>%
        select(-removal_requested_by_supplier) %>%
        replace(is.na(.), "")
    return(meta)
}

meta_to_phy <- function(meta) {
    meta <- sample_data(meta_filt %>% column_to_rownames("npm_research_id"))
    return(meta)
}

meta <- load_metadata("data/SG10K_Health_metadata.n10714.16March2021.parsed.csv", df)

#### Get metadata subsets

In [5]:
subset_metadata <- function(meta, n_subset) {
    meta <- as.data.frame(meta)
    cohorts <- unique(meta$site_supplying_sample)
    subset_vec <- c()

    for (i in cohorts) {
        ids <- meta$npm_research_id[meta$site_supplying_sample == i]
        
        if (length(ids) > n_subset) {
            subset_ids <- sample(ids, n_subset)
            subset_vec <- c(subset_vec, subset_ids)
        } else {
            subset_vec <- c(subset_vec, ids)
        }
    }

    meta_sub <- meta %>%
      filter(npm_research_id %in% subset_vec) 
    return(meta_sub)
}

meta_filt <- subset_metadata(meta, 10)

#### Retrieve rows in abundance matrix

In [6]:
retrieve_rows_from_meta <- function(df, meta) {
    return(df %>% filter(npm_research_id %in% meta$npm_research_id)) 
}

df_filt <- retrieve_rows_from_meta(df, meta_filt)

#### Remove human and unclassified reads

In [7]:
remove_cols <- function(df, col_to_exclude) {
    return(df %>% select(-all_of(col_to_exclude)))
}

df_filt2 <- remove_cols(df_filt, c("Homo sapiens", "unclassified"))

#### Remove taxa present in <~x% of samples

In [8]:
remove_low_freq_taxa <- function(df) {
    n_original <- ncol(df)
    frac_presence <- 0.05
    PA_df <- apply(df[, 2:ncol(df)], 2, function(x) {ifelse(x > 0, T, F)})
    frac_df <- apply(PA_df, 2, function(x) {sum(x) / nrow(PA_df)})
    to_keep <- names(frac_df[frac_df > frac_presence])
    to_keep <- c("npm_research_id", to_keep)
    print(str_glue("{length(to_keep)} / {n_original} taxa are present in {frac_presence} of samples"))
    return(df %>% select(all_of(to_keep)))
}

df_filt3 <- remove_low_freq_taxa(df_filt2)

1576 / 5200 taxa are present in 0.05 of samples


#### Plot metadata barcharts

In [9]:
get_metadata_plots <- function(meta, meta_cols) {
    meta <- as.matrix(meta)
    meta <- as_tibble(meta, rownames = "sample")

    plots <- list()
    
    for (column in meta_cols) {
        plt <- meta %>%
            mutate(across(everything(), as.character)) %>%
            select(all_of(column)) %>%
            group_by_at(column) %>%
            summarise(n = n()) %>%
            ggplot(aes_string(x = column, y = "n", fill = column)) +
                geom_bar(stat = "identity") +
                theme(axis.text.x = element_text(angle = 45, hjust = 1)) +
                geom_text(aes_string(label = "n"))
        plots[[column]] <- plt
    }
    
    return(plots)
}


In [10]:
get_meta_cols <- function(meta) {
    meta_cols <- colnames(meta)[grep("kit|flow_cell|instrument_id|site_supplying", colnames(meta))]
    to_exclude <- c("library_prep_kit", "hiseq_xtm_flow_cell_v2_5_id")
    meta_cols <- meta_cols[!(meta_cols %in% to_exclude)]
    return(meta_cols)
}

meta_cols <- get_meta_cols(meta)

In [11]:
print_group_freqs <- function(meta, meta_cols) {
    for (col in meta_cols) {
        tmp <- tibble(data.frame(meta)) %>%
            group_by(get(col)) %>%
            summarise(n = n())
        print(col)
        print(tmp)
    }
}

print_group_freqs(meta_filt, meta_cols)

[1] "site_supplying_sample"
# A tibble: 7 × 2
  `get(col)`     n
  <chr>      <int>
1 GUSTO         10
2 HELIOS        10
3 MEC           10
4 PRISM         10
5 SERI          10
6 SSMP          10
7 TTSH          10
[1] "extraction_kit"
# A tibble: 6 × 2
  `get(col)`                                         n
  <chr>                                          <int>
1 abGENIX Whole Blood Genomic DNA Extraction Kit    10
2 Chemagic DNA Blood Kit (Perkin Elmer, MA)          8
3 Maxwell RSC Blood DNA Kit (AS1400)                10
4 QIAamp DNA Blood Mini Kit                         10
5 QIAsymphony DSP DNA Midi Kit                      10
6 Unknown                                           22
[1] "instrument_id"
# A tibble: 6 × 2
  `get(col)`     n
  <chr>      <int>
1 NG001         10
2 NG002          6
3 NG003          6
4 NG004          8
5 NG005          7
6 Unknown       33
[1] "hiseq_xtm_sbs_kit_300_cycles_v2__box_1of_2__lot"
# A tibble: 19 × 2
   `get(col)`     n
   <chr>      <int>
 

### Run ANCOM-BC

In [12]:
run_ancom_bc <- function(phy_fn, meta_fn, meta_cols, n_samples, max_iter) {
    result_df <- tibble()

    for (column in meta_cols) {
        print(column)
#         column <- "extraction_kit"
        # Remove levels with < x samples
        tmp <- tibble(data.frame(meta_fn)) %>%
            group_by(get(column)) %>%
            summarise(n = n())
        
        print(tmp)

        # Vector of group levels to keep
        to_keep <- tmp[tmp$n >= n_samples, "get(column)"]$`get(column)`
        to_keep <- to_keep[to_keep != "Unknown"]
        
        print(length(to_keep))

        # Retrieve groupings
        var_values <- sample_data(phy_fn)[[column]]

        # Prune groups
        phy_subset <- prune_samples(var_values %in% to_keep, phy_fn)
        n_filt <- nrow(phy_subset)
        print(str_glue("There are {n_filt} samples after pruning metadata levels"))
        
        out <- ancombc(phyloseq = phy_subset, formula = column,
                      p_adj_method = "BH", zero_cut = 1, lib_cut = 0,
                      group = column, struc_zero = FALSE, neg_lb = FALSE,
                      tol = 1e-5, max_iter = max_iter, conserve = FALSE,
                      alpha = 0.05, global = TRUE)
        
        morsel <- out$res_global %>% mutate(meta_col = column)
        
        result_df <- result_df %>% bind_rows(morsel)
    }
    
    return(result_df %>% rownames_to_column("taxa"))
}

#### Decontamination procedure for each subset

In [13]:
# decontaminate <- function(n) {
    # Load data
#     df <- load_data(str_glue("data/temp_files_{n_subset}/07_abundance_matrix/abundance_matrix.subset_{n_subset}.S.tsv"))
    df <- load_data(str_glue("data/temp_files_9999/07_abundance_matrix/abundance_matrix.subset_9999.S.tsv"))
    meta <- load_metadata("data/SG10K_Health_metadata.n10714.16March2021.parsed.csv", df)
    
    # Get metadata subset
    meta_filt <- subset_metadata(meta, 9999)

    # Filter data
    df_filt <- retrieve_rows_from_meta(df, meta_filt)
    df_filt2 <- remove_cols(df_filt, c("Homo sapiens", "unclassified"))
    df_filt3 <- remove_low_freq_taxa(df_filt2)

    # Get metadata columns of interest
    meta_cols <- get_meta_cols(meta_filt)
    
    # Create phyloseq object
    phy <- phyloseq(df_to_phy(df_filt3), meta_to_phy(meta_filt))

    # Print batch levels and frequencies
#     print_group_freqs(meta_filt, meta_cols)

    # Run ANCOM-BC
    result_df <- run_ancom_bc(phy, meta_filt, meta_cols, n_samples = 3, max_iter = 100)
    result_df
    
#     return(non_contaminants)
#     }


1747 / 5200 taxa are present in 0.05 of samples
[1] "site_supplying_sample"
# A tibble: 7 × 2
  `get(column)`     n
  <chr>         <int>
1 GUSTO           968
2 HELIOS         2285
3 MEC            2787
4 PRISM          1248
5 SERI           1436
6 SSMP             63
7 TTSH            919
[1] 7

[1] "extraction_kit"
# A tibble: 7 × 2
  `get(column)`                                      n
  <chr>                                          <int>
1 abGENIX Whole Blood Genomic DNA Extraction Kit  2281
2 Chemagic DNA Blood Kit (Perkin Elmer, MA)       1040
3 Maxwell RSC Blood DNA Kit (AS1400)              2117
4 QIAamp DNA Blood Mini Kit                        919
5 QIAsymphony DSP DNA Midi Kit                     968
6 QIAsymphony DSP DNA Mini Kit (192)               670
7 Unknown                                         1711
[1] 6

[1] "instrument_id"
# A tibble: 6 × 2
  `get(column)`     n
  <chr>         <int>
1 NG001          1335
2 NG002          1221
3 NG003          1282
4 NG004     

taxa,W,p_val,q_val,diff_abn,meta_col
<chr>,<dbl>,<dbl>,<dbl>,<lgl>,<chr>
Paraburkholderia fungorum...1,2369.5949,0.000000e+00,0.000000e+00,TRUE,site_supplying_sample
Paraburkholderia xenovorans...2,1431.8126,6.262869e-306,8.064137e-306,TRUE,site_supplying_sample
Paraburkholderia hospita...3,660.2795,4.593525e-139,4.828594e-139,TRUE,site_supplying_sample
Ralstonia pickettii...4,3803.0451,0.000000e+00,0.000000e+00,TRUE,site_supplying_sample
Ralstonia insidiosa...5,11006.3824,0.000000e+00,0.000000e+00,TRUE,site_supplying_sample
Ralstonia mannitolilytica...6,4609.9584,0.000000e+00,0.000000e+00,TRUE,site_supplying_sample
Ralstonia solanacearum...7,5302.4572,0.000000e+00,0.000000e+00,TRUE,site_supplying_sample
Burkholderia cepacia...8,4335.3477,0.000000e+00,0.000000e+00,TRUE,site_supplying_sample
Burkholderia sp. LA-2-3-30-S1-D2...9,2308.2847,0.000000e+00,0.000000e+00,TRUE,site_supplying_sample


In [6]:
result_df <- fread("results/decontamination/ANCOMBC_decontamination_results.n9706.100821.csv")
result_df %>%
    separate(taxa, into = c("taxa", NA), remove = T, sep = "\\.\\.") %>%
    pivot_wider(names_from = meta_col, values_from = diff_abn, id_cols = taxa)

hm <- result_df %>%
    separate(taxa, into = c("taxa", NA), remove = T, sep = "\\.\\.") %>%
    ggplot(aes(x = meta_col, y = taxa, fill = diff_abn)) +
    geom_tile() +
    theme(axis.text.y = element_blank(),
          axis.text.x = element_text(angle = 45, hjust = 1)) +
    labs(x = "Batch Info", y = "Species", fill = "Contaminant?")

ggsave("results/decontamination/ANCOMBC_decontamination_heatmap.n9606.100821.png", dpi = 300)

Warning message:
“Expected 2 pieces. Additional pieces discarded in 8 rows [143, 1889, 3635, 5381, 7127, 8873, 10619, 12365].”


taxa,site_supplying_sample,extraction_kit,instrument_id,hiseq_xtm_sbs_kit_300_cycles_v2__box_1of_2__lot,hiseq_xtm_sbs_kit_300_cycles_v2__box_2_of_2__lot,hiseq_xtm_pe_cluster_kit_cbottm_v2__box_1_of_2__lot,hiseq_xtm_pe_cluster_kit_cbottm_v2__box_2_of_2__lot,hiseq_xtm_flow_cell_v2_5_lot
<chr>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>,<lgl>
Paraburkholderia fungorum,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE
Paraburkholderia xenovorans,TRUE,TRUE,FALSE,TRUE,TRUE,TRUE,TRUE,TRUE
Paraburkholderia hospita,TRUE,TRUE,FALSE,TRUE,TRUE,TRUE,TRUE,TRUE
Ralstonia pickettii,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE
Ralstonia insidiosa,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE
Ralstonia mannitolilytica,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE
Ralstonia solanacearum,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE
Burkholderia cepacia,TRUE,TRUE,FALSE,TRUE,TRUE,TRUE,TRUE,TRUE
Burkholderia sp. LA-2-3-30-S1-D2,TRUE,TRUE,FALSE,TRUE,TRUE,TRUE,TRUE,TRUE


Warning message:
“Expected 2 pieces. Additional pieces discarded in 8 rows [143, 1889, 3635, 5381, 7127, 8873, 10619, 12365].”
Saving 6.67 x 6.67 in image



In [18]:
# fwrite(result_df, "results/decontamination/ANCOMBC_decontamination_results.n9706.100821.csv")

In [16]:
dawdawdawdwadwa####################################################

ERROR: Error in eval(expr, envir, enclos): object 'dawdawdawdwadwa' not found


### Explore metadata

In [ ]:
n_subset <- 100

# Load data
df <- load_data(str_glue("data/temp_files_{n_subset}/07_abundance_matrix/abundance_matrix.subset_{n_subset}.S.tsv"))

# Retain relevant metadata
meta <- parse_metadata("data/SG10K_Health_metadata.n10714.16March2021.parsed.csv", df)

# Get metadata columns of interest
meta_cols <- get_meta_cols(meta)

meta <- as.matrix(meta)
meta <- as_tibble(meta, rownames = "sample")

plots <- list()

for (column in meta_cols) {
    plt <- meta %>%
        mutate(across(everything(), as.character)) %>%
        select(all_of(column)) %>%
        group_by_at(column) %>%
        summarise(n = n()) %>%
        ggplot(aes_string(x = column, y = "n", fill = column)) +
            geom_bar(stat = "identity") +
            theme(axis.text.x = element_text(angle = 45, hjust = 1)) +
            geom_text(aes_string(label = "n"))
    plots[[column]] <- plt
}

plots


### MAIN

In [ ]:
# nc_list <- list()

# for (i in c(100)) {
#     nc_list[[str_glue("subset_{i}")]] <- decontaminate(i)
# }

# nc_list

### Sanity check

In [ ]:
# Reduce(intersect, nc_list)

In [ ]:
plot_sanity_plot <- function(X, Y, tax) {
    print(tax)
    temp_plt <- cbind(X, Y) %>%
        mutate(log_abundance = log(get(tax) + 0.0001)) %>%
        ggplot(aes(x = site_supplying_sample, y = get(tax), fill = site_supplying_sample)) + 
        geom_point() +
        geom_boxplot(alpha = 0.5) +
        labs(x = "Source cohort", y = paste(tax, "% abundance")) +
        theme(legend.position = "none",
          axis.text.x = element_text(angle = 45, hjust = 1),
          text = element_text(size = 15))
    return(temp_plt)
}

In [ ]:
# Load data
taxon <- "Cutibacterium acnes"
df <- load_data(str_glue("data/temp_files_{n_subset}/07_abundance_matrix/abundance_matrix.subset_{n_subset}.S.tsv"))

# Remove homo and unclassified
col_to_exclude <- colnames(df)[grepl("Homo|unclassified", colnames(df))]
df <- remove_cols(df, col_to_exclude)

# Retain relevant metadata
meta <- parse_metadata("data/SG10K_Health_metadata.n10714.16March2021.parsed.csv", df)

# Normalise to RA
normalise <- function(df) {
    RA_df <- t(apply(df, 1, function(x) {x / sum(x) * 100}))
    return(as_tibble(RA_df))
}

species_df_RA <- normalise(df)

plot_sanity_plot(df, meta, taxon)
print(str_glue("Max no. of reads = {max(df[, taxon])}"))
print(str_glue("Mean no. of reads = {mean(df[, taxon])}"))

In [ ]:
l1 <- tibble(l1)

fwrite(l1, str_glue("/home/projects/14001280/PROJECTS/blood_microbiome/results/decontamination/decontaminated_list.n{n_subset}.csv"))